In [1]:
import math
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Main Functions

In [2]:
def calV1_3D(N, theta, phi):
  '''
  We calculate the diversity of group opinions using the function calV_3D,
  which yields a value analogous to electric potential energy.
  A higher energy value indicates greater diversity among the group opinions.
  '''
    
  V_max_list = np.array([1.4142135623730951, 3.9481594714802353, 7.666977259633668, 12.49614444572656, 18.511218397802732, 25.619490892577677, 33.89610553620064, 43.30531859956281, 53.851507919120344])
  V_max = V_max_list[N-2]
    
  # Calculate V (mass of external field)
  V = 0
  for i in range(N):
    for j in range(N):
      if i == j:
        continue
      a = [np.sin(theta[i])*np.cos(phi[i]), np.sin(theta[i])*np.sin(phi[i]), np.cos(theta[i])]
      b = [np.sin(theta[j])*np.cos(phi[j]), np.sin(theta[j])*np.sin(phi[j]), np.cos(theta[j])]
      cos_t = np.around(np.dot(a, b),10)
      t = (2-2*cos_t)**0.5
      V += t**0.5

  return 0.5*V/V_max

In [4]:
def MCrun_3D(Vt, V, N, theta, phi):
  # Change one of spins
  i = np.random.randint(N)

  theta_new = np.copy(theta)
  theta_new[i] += 0.005*random.choice([-1, 1])*np.pi
  if theta_new[i] > np.pi:
    theta_new[i] = np.pi - (theta_new[i] - np.pi)
  elif theta_new[i] < 0:
    theta_new[i] = -theta_new[i]

  phi_new = np.copy(phi)
  phi_new[i] += 0.005*random.choice([-1, 1])*np.pi
  if phi_new[i] > np.pi:
    phi_new[i] = -2*np.pi + phi_new[i]
  elif phi_new[i] <= -np.pi:
    phi_new[i] = 2*np.pi + phi_new[i]

  # Calculate the change of potential
  V_new = calV1_3D(N, theta_new, phi_new)
  Vt_init = Vt - V
  Vt_new = Vt - V_new

  # Minimize the potential
  if (abs(Vt_init) > abs(Vt_new)):
    theta = theta_new
    phi = phi_new
    V = V_new

  return V, theta, phi

# Data generation

In [26]:
N = 4
MC_step = 100
ENS_step = 1000

V_tar_lst = [0.1, 0.2, 0.3]
Score_lst = [-0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5]

for V_tar in V_tar_lst:
  for Score_tar in Score_lst:
    S_array = np.zeros((MC_step, ENS_step, N, 3))

    idx = 0
    while idx < ENS_step:
      check = 0
      theta = 1/2*np.pi*np.ones(N)
      phi = 2*np.pi*np.ones(N)
      V = calV1_3D(N, theta, phi)
      for _ in range(1000):
        V, theta, phi = MCrun_3D(V_tar, V, N, theta, phi)
        if abs(calV1_3D(N, theta, phi) - V_tar) < 0.001:
          check = 1
          break
            
      if check:
          for i in np.linspace(0,2,10000):
            S = np.zeros((N, 3))
            th = i*np.pi-theta.mean()
            Ry = np.array([[np.cos(th), 0, np.sin(th)],
                          [0, 1, 0],
                          [-np.sin(th), 0, np.cos(th)]])
            for n in range(N):
              S[n] = Ry@np.array([np.sin(theta[n])*np.cos(phi[n]), np.sin(theta[n])*np.sin(phi[n]), np.cos(theta[n])])

            Score = S[:,2].mean()
            if abs(Score - Score_tar) < 0.001:
              for n in range(N):
                S_array[0,idx,n,:] = S[n]
              idx += 1
              if idx == ENS_step:
                    break

    with open(f"Sarray_N{N}_Score{Score_tar}_Vt{V_tar}.pkl","wb") as f:
      pickle.dump(S_array, f)

In [27]:
N = 4
MC_step = 100
ENS_step = 1000

V_tar_lst = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
Score_lst = [-0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5]

for V_tar in V_tar_lst:
  for Score_tar in Score_lst:
    S_array = np.zeros((MC_step, ENS_step, N, 3))

    idx = 0
    while idx < ENS_step:
      check = 0
      theta = 1/2*np.pi*np.random.rand(N)
      phi = 2*np.pi*np.random.rand(N)
      V = calV1_3D(N, theta, phi)
      for _ in range(1000):
        V, theta, phi = MCrun_3D(V_tar, V, N, theta, phi)
        if abs(calV1_3D(N, theta, phi) - V_tar) < 0.001:
          check = 1
          break
            
      if check:
          for i in np.linspace(0,2,10000):
            S = np.zeros((N, 3))
            th = i*np.pi-theta.mean()
            Ry = np.array([[np.cos(th), 0, np.sin(th)],
                          [0, 1, 0],
                          [-np.sin(th), 0, np.cos(th)]])
            for n in range(N):
              S[n] = Ry@np.array([np.sin(theta[n])*np.cos(phi[n]), np.sin(theta[n])*np.sin(phi[n]), np.cos(theta[n])])

            Score = S[:,2].mean()
            if abs(Score - Score_tar) < 0.001:
              for n in range(N):
                S_array[0,idx,n,:] = S[n]
              idx += 1
              if idx == ENS_step:
                    break

    with open(f"Sarray_N{N}_Score{Score_tar}_Vt{V_tar}.pkl","wb") as f:
      pickle.dump(S_array, f)